In [73]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
import pandas as pd
from tensorflow import keras

processed_data = np.load('Full_array.npy')
#The size of the dataset is 10,000; but I only filled the first 9166 with values, the rest are zeros. (shouldn't be counted)
number = 9166
processed_data = processed_data[0:number]

np.random.seed(1)
tf.set_random_seed(1)

In [74]:
from PIL import Image
from cv2 import resize
from sklearn.utils import shuffle

In [75]:
#we want to normalize our iamge data to go from 0 to 1
#processed_data = processed_data.reshape(np.size(processed_data))
#processed_data = (processed_data - min(processed_data))/ (max(processed_data) - min(processed_data))
#processed_data = processed_data.reshape(number,28,28)
#I actually am not sure how it is scaled. however, some of the pixels are negative for some reason???

#also, this did not help.

In [76]:
#hyperparameters:
HP_allowed_confidence = 0.98
test_train_split = 0.85
batch_size = 128
epoch_number = 100
learning_rate = 0.01

#I have a non-round number of examples
train_split_indice = int(np.round(test_train_split*number))

In [77]:
#Load in my dataset of targets, targets are strings labels under the name "Class"
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[2,3,4,8,15,21,27], nrows=number)
#galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", nrows=number)
Class = galaxyzoo["gz2class"].values
RA = galaxyzoo['ra'].values
DEC = galaxyzoo['dec'].values
Spiral = galaxyzoo['t01_smooth_or_features_a02_features_or_disk_debiased'].values
Elliptical = galaxyzoo['t01_smooth_or_features_a01_smooth_debiased'].values
Anythingelse = galaxyzoo['t01_smooth_or_features_a03_star_or_artifact_debiased'].values

In [78]:
#I want to take the first character of the Class string and interpret as a integer, ala MNIST example code
dictionary = {'A':int(2),'E':int(1),'S':int(0)}

In [79]:
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [80]:
#go through, find the arrays of zero, set that target to 'A' = 2
for i in range(len(target)):
    if processed_data[i].any() == 0:
        target[i] = 2

In [81]:
#split my data between training and test sets
train_target = target[0:train_split_indice]
test_target = target[train_split_indice:number]
train_images = processed_data[0:train_split_indice]
test_images = processed_data[train_split_indice:number]

In [82]:
#Create 3 more images from each of my training and testing images that is each image but flipped 90 degrees...
train_images_flip1 = np.flip(train_images,1)
train_images_flip2 = np.flip(train_images,2)
train_images_flip3 = np.flip(train_images_flip1,2)

flipped_img = np.append(train_images, train_images_flip1, 0)
flipped_img = np.append(flipped_img, train_images_flip2, 0)
flipped_img = np.append(flipped_img, train_images_flip3, 0)

flipped_tar = np.append(train_target,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)

print(np.shape(flipped_tar))
print(np.shape(flipped_img))
print(np.size(train_target)*4)

print(np.shape(test_target))

(31164,)
(31164, 28, 28)
31164
(1375,)


In [83]:
train_images = flipped_img
train_target = flipped_tar

In [84]:
np.shape(train_images[0])

(28, 28)

In [85]:
"""
#rotate each image in the target_images by a random amount
rotations_array = np.empty((31164,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)


#every time we run this segment, we increase our training set by a factor of 2. this means it will train in 120 * number of times
# times 20 epochs = a long time.
"""

'\n#rotate each image in the target_images by a random amount\nrotations_array = np.empty((31164,28,28))\nfor i in range(np.shape(train_images)[0]):\n    degree = np.random.uniform(-45,45,1)\n    img = Image.fromarray(train_images[i])\n    rot = img.rotate(degree)\n    rot = np.asarray(rot)\n    rotations_array[i] = rot\n#now append rotations_array to train_images\ntrain_images = np.append(train_images, rotations_array,0)\n#now append the class labels to the train_target array\ntrain_target = np.append(train_target,train_target)\n\n\n#every time we run this segment, we increase our training set by a factor of 2. this means it will train in 120 * number of times\n# times 20 epochs = a long time.\n'

In [86]:
def cv2_clipped_zoom(img, zoom_factor):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

In [87]:
"""
#increase by a factor of two again, like above
rotations_array = np.empty((31164*2,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    zoom_factor = np.random.uniform(0.7,1.3,1)
    rot = cv2_clipped_zoom(np.asarray(rot),zoom_factor)
    #rot = rot.resize()
    #rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)
"""

'\n#increase by a factor of two again, like above\nrotations_array = np.empty((31164*2,28,28))\nfor i in range(np.shape(train_images)[0]):\n    degree = np.random.uniform(-45,45,1)\n    img = Image.fromarray(train_images[i])\n    rot = img.rotate(degree)\n    zoom_factor = np.random.uniform(0.7,1.3,1)\n    rot = cv2_clipped_zoom(np.asarray(rot),zoom_factor)\n    #rot = rot.resize()\n    #rot = np.asarray(rot)\n    rotations_array[i] = rot\n#now append rotations_array to train_images\ntrain_images = np.append(train_images, rotations_array,0)\n#now append the class labels to the train_target array\ntrain_target = np.append(train_target,train_target)\n'

In [88]:
#we want to form a subset of our training examples that are probabilities that they are sorted correctly greater than %90
#we will then train this dataset additional times.
train_images_number = np.shape(train_images)[0]
test_images_number = np.shape(test_images)[0]
train_images = train_images.reshape(train_images_number,28,28,1)
test_images = test_images.reshape(test_images_number,28,28,1)

In [89]:
#shuffle the order of the augmented images randomly, shuffle the order of the targets the same way.
train_images, train_target = shuffle(train_images, train_target, random_state=1)

In [90]:
#Seems I need to pad the arrays with zeros to get to 32x32, then copy the values of the first monochromatic channel into 3 channels
#to use tranfer learning
print(np.shape(train_images))
"""
Lets investiagte how to manipulate our data using padding and appending to copy make our arrays into 32,32,3 shapes so our network
can handle them


zero_array = np.zeros((28,28,1))
zero_array=np.pad(zero_array,((2,2),(2,2),(0,0)),'edge')
print(np.shape(zero_array))


zero_array_new = zero_array
zero_array_new = np.append(zero_array_new,zero_array,2)
zero_array_new = np.append(zero_array_new,zero_array,2)
print(np.shape(zero_array_new))

neat now implement on our actual sets of data
"""
train_images = np.pad(train_images,((0,0),(2,2),(2,2),(0,0)),'constant', constant = 0)
test_images = np.pad(test_images,((0,0),(2,2),(2,2),(0,0)),'constant', constant = 0)

train_images_middle_step = np.append(train_images, train_images, 3)
train_images = np.append(train_images_middle_step, train_images, 3)

test_images_middle_step = np.append(test_images,test_images, 3)
test_images = np.append(test_images_middle_step,test_images, 3)

print(np.shape(test_images))
print(np.shape(train_images))

#consider saving this array in the future instead of wasting time re-running the damn thing.

(31164, 28, 28, 1)
(1375, 32, 32, 3)
(31164, 32, 32, 3)


In [101]:
#load in and define my model, using transfer learning from a NN that was trained on image net
#https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e
from keras.applications import vgg16
from keras import layers
#Input_layer = layers.Input(28,28,3)
base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(32,32,3))

x=base_model.output
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=layers.Dense(512,activation='relu')(x) #dense layer 2
x=layers.Dense(128,activation='relu')(x) #dense layer 3
preds=layers.Dense(3,activation='softmax')(x) #final layer with softmax activation

In [102]:
#define our model using the loaded base_model and our added layers
from keras import Model
model=Model(inputs=base_model.input,outputs=preds)

In [103]:
#now that we have a model, we want to check the number of layers
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_4
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_3
20 dense_9
21 dense_10
22 dense_11
23 dense_12


In [104]:
#using the number of layers, we want to then tell the classifier to only train the new top layers, the dense layers we added
#so that we maximize the benefit of the imagenet training.

#for layer in model.layers:
#    layer.trainable=False

# or if we want to set the first 19 layers of the network to be non-trainable
for layer in model.layers[:19]:
    layer.trainable=False
for layer in model.layers[19:]:
    layer.trainable=True

In [105]:
#set up a validation and training set to tune hyperparameters.
validation_split = int(np.round(np.size(train_target)*0.90))
validation_images = train_images[validation_split::]
validation_target = train_target[validation_split::]
train_target = train_target[0:validation_split]
train_images = train_images[0:validation_split]

In [106]:
#define an optimizer, loss, and accuracy metric.
adam = tf.train.AdamOptimizer(learning_rate = learning_rate)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [107]:
#train, then print validation scores
for i in range(epoch_number):
    model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True)
    val_loss, val_acc = model.evaluate(validation_images, validation_target)
    print(i)
    print('Test accuracy:', val_acc)

Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
0
Test accuracy: 0.802139037475654
Epoch 1/1
2805/2805 [==============================] - 38s 14ms/step
1
Test accuracy: 0.8135472371403971
Epoch 1/1
2805/2805 [==============================] - 38s 14ms/step
2
Test accuracy: 0.8017825312792937
Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
3
Test accuracy: 0.7878787880062844
Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
4
Test accuracy: 0.8153297683346931
Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
5
Test accuracy: 0.8256684492828587
Epoch 1/1
2805/2805 [==============================] - 41s 15ms/step
6
Test accuracy: 0.8163992870512705
Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
7
Test accuracy: 0.8253119430439995
Epoch 1/1
2805/2805 [==============================] - 39s 14ms/step
8
Test accuracy: 0.8295900179315591
Epoch 1/1
2805/2805 [==============================] - 3

2805/2805 [==============================] - 39s 14ms/step
41
Test accuracy: 0.8406417113149443
Epoch 1/1
2805/2805 [==============================] - 38s 14ms/step
42
Test accuracy: 0.8402852050760851
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
43
Test accuracy: 0.8452762924626124
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
44
Test accuracy: 0.849197861068814
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
45
Test accuracy: 0.8488413548512043
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
46
Test accuracy: 0.8402852050973345
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
47
Test accuracy: 0.8467023174180491
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
48
Test accuracy: 0.8402852050973345
Epoch 1/1
2805/2805 [==============================] - 37s 13ms/step
49
Test accuracy: 0.8463458111579405


In [98]:
for i in range(epoch_number):
    model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True)
    val_loss, val_acc = model.evaluate(validation_images, validation_target)
    print(i+50)
    print('Test accuracy:', val_acc)

Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
50
Test accuracy: 0.8472400514243962
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
51
Test accuracy: 0.8292682927594411
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
52
Test accuracy: 0.8456354299619966
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
53
Test accuracy: 0.8459563543003851
Epoch 1/1
3116/3116 [==============================] - 45s 14ms/step
54
Test accuracy: 0.8456354300385109
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
55
Test accuracy: 0.845314505853151
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
56
Test accuracy: 0.8427471116816432
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
57
Test accuracy: 0.8369704748913933
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
58
Test accuracy: 0.8427471117581574
Epoch 1/1
3116/3116 [==========================

3116/3116 [==============================] - 42s 14ms/step
91
Test accuracy: 0.8687419769699675
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
92
Test accuracy: 0.8677792041843448
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
93
Test accuracy: 0.8636071887799803
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
94
Test accuracy: 0.8626444158413291
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
95
Test accuracy: 0.8626444159943577
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
96
Test accuracy: 0.858151476328119
Epoch 1/1
3116/3116 [==============================] - 45s 14ms/step
97
Test accuracy: 0.8677792041843448
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
98
Test accuracy: 0.8681001282931905
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
99
Test accuracy: 0.8677792040313163


In [99]:
for i in range(epoch_number):
    model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True)
    val_loss, val_acc = model.evaluate(validation_images, validation_target)
    print(i+50)
    print('Test accuracy:', val_acc)

Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
50
Test accuracy: 0.8693838254937158
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
51
Test accuracy: 0.8616816431322208
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
52
Test accuracy: 0.8716302953268352
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
53
Test accuracy: 0.8591142491137416
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
54
Test accuracy: 0.8681001284462191
Epoch 1/1
3116/3116 [==============================] - 48s 15ms/step
55
Test accuracy: 0.877727856302445
Epoch 1/1
3116/3116 [==============================] - 42s 14ms/step
56
Test accuracy: 0.8629653402562318
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
57
Test accuracy: 0.871309371064961
Epoch 1/1
3116/3116 [==============================] - 47s 15ms/step
58
Test accuracy: 0.8543003851856286
Epoch 1/1
3116/3116 [===========================

3116/3116 [==============================] - 42s 14ms/step
91
Test accuracy: 0.8687419768934531
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
92
Test accuracy: 0.8626444159943577
Epoch 1/1
3116/3116 [==============================] - 44s 14ms/step
93
Test accuracy: 0.8828626444924321
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
94
Test accuracy: 0.876123234993074
Epoch 1/1
3116/3116 [==============================] - 44s 14ms/step
95
Test accuracy: 0.8693838254937158
Epoch 1/1
3116/3116 [==============================] - 44s 14ms/step
96
Test accuracy: 0.8706675225412126
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
97
Test accuracy: 0.8632862645181061
Epoch 1/1
3116/3116 [==============================] - 44s 14ms/step
98
Test accuracy: 0.87997432605905
Epoch 1/1
3116/3116 [==============================] - 43s 14ms/step
99
Test accuracy: 0.8729139922978177


In [108]:
#model.summary()
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)

1375/1375 [==============================] - 18s 13ms/step
Test accuracy: 0.8050909084840254
